In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

from IPython.display import display, Markdown

from pymedphys.msq import (
    multi_mosaiq_connect, get_qcls_by_date,
    mosaiq_connect, get_staff_name)

In [ ]:
start = '2019-11-01 00:00:00'
end = '2019-12-01 00:00:00'

In [ ]:
centres = ['rccc', 'nbcc', 'sash']

servers = {
    'rccc': 'msqsql', 
    'nbcc': 'physics-server:31433',
    'sash': 'physics-server'
}

servers_list = [
    item for _, item in servers.items()
]

physics_location = {
    'rccc': 'Physics_Check',
    'nbcc': 'Physics',
    'sash': 'Physics_Check'
}

imrt_task_names = {
    'nbcc': ['Physics Check IMRT'],
    'rccc': ['IMRT Physics Check']
}

non_imrt_task_names = {
    'nbcc': ['Physics Check 3DCRT'],
    'rccc': ['3D Physics Check', 'Electron Factor']
}



sash_physics_task_name = 'Physics QA           '

In [ ]:
# with multi_mosaiq_connect(servers_list) as cursors:
#     for centre in centres:
#         display(Markdown('### {}'.format(centre)))
#         cursor = cursors[servers[centre]]
#         display(get_staff_name(cursor, physics_ids[centre]))

In [ ]:
# Working out physics_id

# with mosaiq_connect(servers['sash']) as cursor:
#     display(get_qcls_by_date(cursor, start, end))
    

In [ ]:
with multi_mosaiq_connect(servers_list) as cursors:
    
    results = {
        centre: get_qcls_by_date(
            cursors[servers[centre]], physics_location[centre], start, end
        )
        for centre in centres
    }

In [ ]:
for server in servers:
    results[server] = results[server].drop_duplicates(subset='actual_completed_time', keep='first')

In [ ]:
for server in servers:
    display(Markdown("### {}".format(server)))
    
    display(results[server])

In [ ]:
def count_results(results, imrt_task_names, non_imrt_task_names, server):
    
    imrt_results = 0
    non_imrt_results = 0
    
    for task in results[server]['task']:
        trimmed_task = task.strip()
        if trimmed_task in imrt_task_names[server]:
            imrt_results = imrt_results + 1
        elif trimmed_task in non_imrt_task_names[server]:
            non_imrt_results = non_imrt_results + 1
        else:
            print(trimmed_task)
    
    
    return {
        'imrt_results': imrt_results,
        'non_imrt_results': non_imrt_results
    }

In [ ]:
counts = {
    server: count_results(results, imrt_task_names, non_imrt_task_names, server)
    for server in ['rccc', 'nbcc']
}

counts

In [ ]:
# SASH results
len(results['sash']['task'])